## Driver Drowsiness Detection System

- The system capture live footage of user using camera connected to the device.
- The system will detect the face, left eye, and right eye of the user.
- After detecting face, left eye and right eye of user, the model classifier in the system will classify if user is drowsy by classifying if both left eye and right eye of user is closed or opened.
- User left eye and right eye is closed when model classifier in system classifies as "0"; user left eye and right eye is opened when model model classifier in system classifies as "1"
- Score which is a value to keep track how long user has closed his/her eyes will increase if user's eyes is closed and will decrease if user's eyes is opened until the score is 0.
- When score goes above 18, an alarm will sound and the application window will have a red rectangle blinking around the frame of the live footage.
- The system will capture the an image of user that is in drowsy state when the score goes above 18 save the image in the same folder as the system file. File name of image is "user_drowsy_image.jpg".
- User can exit the system by click the "q" button on their keyboard.

In [1]:
import os
import numpy as np

#import opencv(cv2) library for computer vision 
import cv2

#import keras library for artificial neural networks or model trained with deep learning
from keras.models import load_model

#python module pygame import mixer module for loading and playing audio file
from pygame import mixer

#initialize the mixer module
mixer.init()

#load the alarm audio file with mixer module according to directory of audio file
sound = mixer.Sound('Drowsiness detection/alarm.wav')

#load pre-trained model for face detection, left eye detection and right eye detection
face = cv2.CascadeClassifier('Drowsiness detection/haar cascade files\haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier('Drowsiness detection/haar cascade files\haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('Drowsiness detection/haar cascade files\haarcascade_righteye_2splits.xml')

#load pre-trained deep learning model for drowsiness detection
model = load_model('Drowsiness detection/models/cnncat2.h5')

#get current working directory of a process or current directory of file
path = os.getcwd()

#instantiate a object for video capture module of OpenCV library for video capture live footage from primary camera device
cap = cv2.VideoCapture(0)

#instantiate a font from OpenCV library
font = cv2.FONT_HERSHEY_TRIPLEX

#initialize varaible for score, thicc(thickness of rectangle border) and array to store the state of left eye and right eye
score=0
thicc=2
rpred=[99]
lpred=[99]

#capture live video footage from camera and detect user's face, left eye and right eye and determine if the user is drowsy or not
while(True):
    #get video footage captured by camera and initialize variable to store each frame of video footage
    ret, frame = cap.read()
    height,width = frame.shape[:2]
    #convert RGB/BGR colour space to grayscale for image segmentation
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #apply pre-defined model to detect face of person to create region of interest
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    #apply pre-defined model to detect left eye and right eye of person from region of interest generated
    left_eye = leye.detectMultiScale(gray)
    right_eye = reye.detectMultiScale(gray)
    #draw a rectangle on the frame/video footage as video footage captured by camera is displayed on application as
    #the background of the text
    cv2.rectangle(frame, (0,height-50) , (280,height) , (0,0,0) , thickness=cv2.FILLED )
    
    #create rectangle on the area of face of person that is detected by the classifier
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 3)
        
    #detect right eye of person from the region of interest generated which is the face of user    
    for (x,y,w,h) in right_eye:
        #draw rectangle around the right eye that is detected from the face of person
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 2)
        #get right eye of person
        r_eye=frame[y:y+h,x:x+w]
        #convert rgb colour space of right eye image to grayscale
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        #resize right eye image to 24*24 pixels
        r_eye = cv2.resize(r_eye,(24,24))
        #normalize the image of right eye data to be between value 0-1
        r_eye= r_eye/255
        r_eye= r_eye.reshape(24,24,-1)
        r_eye = np.expand_dims(r_eye,axis=0)
        #predict if right eye is closed using pre-trained CNN model
        rpred = model.predict_classes(r_eye)
        break
        
    #detect left eye of person from the region of interest generated which is the face of user  
    for (x,y,w,h) in left_eye:
        #draw rectangle around the left eye that is detected from the face of person
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 2)
        #get left eye of person
        l_eye=frame[y:y+h,x:x+w]
        #convert rgb colour space of left eye image to grayscale
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)
        #resize right eye image to 24*24 pixels
        l_eye = cv2.resize(l_eye,(24,24))
        #normalize the image of right eye data to be between value 0-1
        l_eye= l_eye/255
        l_eye=l_eye.reshape(24,24,-1)
        l_eye = np.expand_dims(l_eye,axis=0)
        #predict if left eye is closed using pre-trained CNN model
        lpred = model.predict_classes(l_eye)
        break
        
    """if pre-trained CNN model detected and determined both left eye and right eye is closed or predicted as "0", 
    score increases and text displayed on application window will change to "Closed" """
    if(rpred[0]==0 and lpred[0]==0):
        if score < 30:
            score= score+1
        text = "Closed"
    #if pre-trained CNN model detected and determined both left eye and right eye is open or predicted as "1", 
    #score decreases and text displayed on application window will change to "Open"
    else:
        score= score-1
        text = "Open"
    #change score back to zero if score decreases below zero
    if(score<0):
        score=0
    #display "Score: " on application windows to display score of left eye and right eye to determine if user is drowsy and fall asleep
    cv2.putText(frame, text +' Score:'+str(score),(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
    #when score is over 18, determines that user is drowsy and falling asleep while driving
    if(score>18):
        #person is feeling sleepy and the alarm audio is sounded to wake user
        try:
            sound.play()
        except: # isplaying = False
            pass
        #increase thickness of red border that flashes around frame of captured live video footage when the system detect user falling asleep
        if(thicc<16):
            thicc= thicc+2
        #decreases thickness of red border that flashes around frame of captured live video footage to stop border thickness from increasing
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        #draw rectangle border around entire frame of live video footage captured by camera if user is drowsy and fallin asleep 
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc)
        #capture image of user when user system determine user is drowsy
        if(score == 19 and rpred[0]==0 and lpred[0]==0):
            cv2.imwrite(os.path.join(path,'user_drowsy_image.jpg'),frame)
            
    #display frame of live video footage captured by camera
    cv2.imshow('frame',frame)
    
    #close/stop application showing live footage captured by camera when the key "q" is entered
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
#interrupt stop capturing video footage from camera and close/destroy the application windowdisplaying captured live footage
cap.release()
cv2.destroyAllWindows()

pygame 2.0.1 (SDL 2.0.14, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
